# Import Gaia data and make a luminosity map

*The main table is gaiadr2.gaia_source, and [here](http://gea.esac.esa.int/archive/documentation/GDR2/Gaia_archive/chap_datamodel/sec_dm_main_tables/ssec_dm_gaia_source.html) is the description of the columns*

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import astropy.units as u
from astropy.coordinates import ICRS
from astropy.coordinates.sky_coordinate import SkyCoord
from astropy.io.votable import parse
from astroquery.gaia import Gaia
import numpy as np
import vespa


Created TAP+ (v1.0.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: False
	Port: 80
	SSL Port: 443


/Users/ageller/anaconda/envs/py36/lib/python3.6/site-packages/numpy/lib/npyio.py:2222: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  output = genfromtxt(fname, **kwargs)
/Users/ageller/anaconda/envs/py36/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/ageller/anaconda/envs/py36/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module

### Retrieve the data 

*Note: The query to the archive is with ADQL (Astronomical Data Query Language). *

In [ ]:
#NOTE: not all sources have RVs
#cmd = "SELECT TOP 100 ra, dec, parallax, pmra, pmdec, radial_velocity FROM gaiadr2.gaia_source"

#NOTE: It looks like there are 
#480235 sources with g <=10 and with parallaxes
#1236351 sources with g <=11 and with parallaxes
#I could only get 3000000 for g<=12.  I wonder if that is the limit?
#I think this is already ordered by gmag, so by default this will pull the brightest first
N = 1e7
gmin = 11
gmax = 24

#cmd = "SELECT TOP " + str(np.int(N))+" \
cmd = "SELECT \
ra, dec, parallax, teff_val, lum_val, phot_g_mean_mag, a_g_val  \
FROM gaiadr2.gaia_source \
WHERE phot_g_mean_mag<=" + str(gmax) +" \
AND phot_g_mean_mag>=" + str(gmin) +" \
AND phot_g_mean_mag IS NOT NULL \
AND parallax IS NOT NULL "#\
#AND radial_velocity IS NOT NULL"
print(cmd)

#asynchronous commands for larger files
job = Gaia.launch_job_async(cmd, dump_to_file=True)

print (job)

In [3]:
#r = job.get_results()

vot = parse("LSST-result.vot")
table = vot.get_first_table()
r = table.array

In [19]:
print(r)

print(table)


[(249.42349063804585, -38.266628527080016, 0.4594311390035986, --, --, 19.87489128112793, --)
 (249.7339378042595, -38.15464728562749, -0.2518601825402249, --, --, 19.567447662353516, --)
 (249.73787701668542, -38.098100666684694, 0.8338294682230369, --, --, 19.453121185302734, --)
 ...
 (283.5690969437813, -16.108083824587247, 0.09002235746694323, 4365.25, --, 16.606199264526367, 0.7813000082969666)
 (283.1365662278172, -16.126315956308563, 0.6192785688518824, --, --, 18.804548263549805, --)
 (283.39064870775286, -16.07743069188592, 1.0233496900961845, --, --, 19.015169143676758, --)]
        ra                 dec         ... phot_g_mean_mag a_g_val
       deg                 deg         ...       mag         mag  
------------------ ------------------- ... --------------- -------
249.42349063804585 -38.266628527080016 ...       19.874891      --
 249.7339378042595  -38.15464728562749 ...       19.567448      --
249.73787701668542 -38.098100666684694 ...       19.453121      --
248.1

### Convert these ra, dec, parallax coordinates to 3D cartesian

*To match the input that I would get from Katie's model*

In [17]:
#force the parallax to be positive!
dist = np.abs((r['parallax']).to(u.parsec, equivalencies=u.parallax()))
astroC = ICRS(ra=r['ra'], dec=r['dec'], distance=dist)
print(astroC.cartesian)


AttributeError: 'MaskedArray' object has no attribute 'to'

### Estimate the absolute G mag, given the apparent and distance

*This does not (yet) account for redenning.*

In [ ]:
MbolSun = 4.73
absG = r['phot_g_mean_mag'].values - 5.*np.log10(dist.to(u.parsec).value/10.) - r['a_g_val'].values

L12G = 10.**(-0.4*(absG - MbolSun)) #luminosity in the G band (not bolometric), in LSun.  Is this correct?

print(r['phot_g_mean_mag'])
print(absG)
print(L12G)